In [5]:
from datasets import load_dataset, DatasetDict


In [6]:
common_voice=load_dataset('common_voice.hf')

Resolving data files: 100%|██████████| 22/22 [00:00<00:00, 169311.35it/s]


In [7]:
common_voice

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 39357
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 10547
    })
})

In [8]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [10]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-medium", language="Persian", task="transcribe",cache_dir='v3')

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

/home/user/.pyenv/versions/3.10.13/lib/python3.10/site-packages/transformers/utils/hub.py:122: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [11]:
import evaluate

metric = evaluate.load("wer")

In [12]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [13]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium",cache_dir='v3')

config.json: 100%|██████████| 1.99k/1.99k [00:00<00:00, 3.66MB/s]
model.safetensors: 100%|██████████| 3.06G/3.06G [18:31<00:00, 2.75MB/s]
generation_config.json: 100%|██████████| 3.72k/3.72k [00:00<00:00, 9.28MB/s]


In [14]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [18]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-large-medium-fa",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [19]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [24]:
%env CUDA_VISIBLE_DEVICES=0,1,3,5

trainer.train()

env: CUDA_VISIBLE_DEVICES=0,1


Signature:
trainer.train(
    resume_from_checkpoint: Union[str, bool, NoneType] = None,
    trial: Union[ForwardRef('optuna.Trial'), Dict[str, Any]] = None,
    ignore_keys_for_eval: Optional[List[str]] = None,
    **kwargs,
)
Docstring:
Main training entry point.

Args:
    resume_from_checkpoint (`str` or `bool`, *optional*):
        If a `str`, local path to a saved checkpoint as saved by a previous instance of [`Trainer`]. If a
        `bool` and equals `True`, load the last checkpoint in *args.output_dir* as saved by a previous instance
        of [`Trainer`]. If present, training will resume from the model/optimizer/scheduler states loaded here.
    trial (`optuna.Trial` or `Dict[str, Any]`, *optional*):
        The trial run or the hyperparameter dictionary for hyperparameter search.
    ignore_keys_for_eval (`List[str]`, *optional*)
        A list of keys in the output of your model (if it is a dictionary) that should be ignored when
        gathering predictions for evaluatio

In [22]:
import torch
torch.cuda.empty_cache()